<a href="https://colab.research.google.com/github/TheG00dB0y/traffic-filter/blob/master/create_dataset_for_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Widerface data preprocessing




In [2]:
from IPython.display import clear_output
from IPython.display import Image

import cv2
import os
import random
from shutil import copyfile
from google.colab.patches import cv2_imshow
clear_output()

Copy widerface dataset (if not downloaded :
[Dataset download page](http://shuoyang1213.me/WIDERFACE/))

In [6]:
!cp "/content/drive/My Drive/Train_images/WIDER_test.zip" "/content/wider_test.zip" 
!unzip wider_test.zip
!rm wider_test.zip -r

!cp "/content/drive/My Drive/Train_images/WIDER_train.zip" "/content/wider_train.zip" 
!unzip wider_train.zip
!rm wider_train.zip -r

!cp "/content/drive/My Drive/Train_images/WIDER_val.zip" "/content/wider_val.zip" 
!unzip wider_val.zip
!rm wider_val.zip -r
clear_output()

In [7]:
#Download annotations
!curl -L "http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/support/bbx_annotation/wider_face_split.zip" > annotations.zip; unzip annotations.zip; rm annotations.zip
clear_output()

Set up empty folder structure if not exists

In [12]:
if not os.path.exists('train'):
    os.makedirs('train')
if not os.path.exists('train/images'):
    os.makedirs('train/images')
    os.makedirs('train/labels')
if not os.path.exists('valid'):
    os.makedirs('valid')
if not os.path.exists('valid/images'):
    os.makedirs('valid/images')
    os.makedirs('valid/labels')
            

Adding files to colab


**train set**

In [13]:
img_source_dir = "/content/WIDER_train/images"
lines = open("/content/wider_face_split/wider_face_train_bbx_gt.txt").readlines()
img_output_dir = "train/images"
label_output_dir = "train/labels"

**Test Set (added with train set)**

In [15]:
img_source_dir = "/content/WIDER_test/images"
lines = open("/content/wider_face_split/wider_face_test_filelist.txt").readlines()
img_output_dir = "train/images"
label_output_dir = "train/labels"

**Validation Set**

In [18]:
img_source_dir = "/content/WIDER_val/images"
lines = open("/content/wider_face_split/wider_face_val_bbx_gt.txt").readlines()
img_output_dir = "valid/images"
label_output_dir = "valid/labels"

**Run for each sets**

In [19]:
for line in lines:
  if line.endswith(".jpg\n"):
    box = []
    line = line.strip()
    line_list = line.split('/',1)
    img_name = line_list[1].strip()
    img_name_label = img_name.split(".jpg")[0]
    try:
      img = cv2.imread(os.path.join(img_source_dir,line))
      img_h, img_w , _ = img.shape
      copyfile(os.path.join(img_source_dir,line),os.path.join(img_output_dir,img_name))
    except:
      print("ERROR")
      continue

  elif(len(line)>10):
      line = line.split()
      # print(line)
      # print(os.path.join('train/labels',img_name_label + '.txt'))
      # label_file = open(os.path.join('train/labels',img_name_label + '.txt'),"w") 
      #convert labels
      [x1,y1,w,h] = line[:4]
      # print(x1,y1,w,h)
      x = (int(x1)  + int(w)/2 ) /img_w
      y = (int(y1)  + int(h)/2 ) /img_h
      w = int(w) / img_w
      h = int(h) / img_h
      # print(x,y,w,h)
      label_line = '11 ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h)
      # print(label_line)
      with open(os.path.join(label_output_dir,img_name_label + '.txt'),"a") as label_file:
        label_file.write(label_line)
# clear_output()

Display image

In [ ]:
# !ls "/content/train/images"
from IPython.display import Image
Image('/content/train/images/9_Press_Conference_Press_Conference_9_38.jpg')
# cv2_imshow('/content/train/images/9_Press_Conference_Press_Conference_9_38.jpg')

#Dataset for udacity self driving cars

In [20]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
#temporary until bug is fixed in current master
!git reset --hard 5ba1de0cdcc414c69ceb7a4c45eb1e3895eca32a
%cd /content

!pip install -r yolov5/requirements.txt  # install dependencies
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.5.1+cu101 CPU


In [ ]:
# %cd /content/yolov5
# !bash "/content/yolov5/weights/download_weights.sh"

In [ ]:
%cd /content
!curl -L "https://public.roboflow.ai/ds/XCnZS0hI6C?key=H6G2RBhLy3" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
clear_output()
print("Dataset downloaded")

Add new classes in the config

In [ ]:
%cat data.yaml

In [23]:


import os
import random
from shutil import copyfile

 
img_source_dir = "/content/export/images"
label_source_dir = "/content/export/labels"
train_size = 0.8
if not (isinstance(img_source_dir, str)):
    raise AttributeError('img_source_dir must be a string')
    
if not os.path.exists(img_source_dir):
    raise OSError('img_source_dir does not exist')
    
if not (isinstance(train_size, float)):
    raise AttributeError('train_size must be a float')
        
    # Set up empty folder structure if not exists
if not os.path.exists('train'):
    os.makedirs('train')
if not os.path.exists('train/images'):
    os.makedirs('train/images')
    os.makedirs('train/labels')
if not os.path.exists('valid'):
    os.makedirs('valid')
if not os.path.exists('valid/images'):
    os.makedirs('valid/images')
    os.makedirs('valid/labels')
            

train_counter = 0
validation_counter = 0



In [24]:
import cv2
# Randomly assign an image to train or validation folder
for filename in os.listdir(img_source_dir):
    if filename.endswith(".jpg"):
        fileparts = filename.split('.jpg')
        # print(os.path.join(label_source_dir,fileparts[0] + '.txt'))
        # print(os.path.join('train/images',filename))
        # image = cv2.imread(os.path.join(img_source_dir,filename)) 
        # image = cv2.resize(image, (412, 412)) 
        if random.uniform(0, 1) <= train_size:

            # copyfile(image, os.path.join('train/images',filename))
            copyfile(os.path.join(img_source_dir,filename), os.path.join('train/images',filename))
            copyfile(os.path.join(label_source_dir,fileparts[0] + '.txt'), os.path.join('train/labels',fileparts[0] + '.txt'))
            train_counter += 1
        else:
            copyfile(os.path.join(img_source_dir,filename), os.path.join('valid/images',filename))
            copyfile(os.path.join(label_source_dir,fileparts[0] + '.txt'), os.path.join('valid/labels',fileparts[0] + '.txt'))
            validation_counter += 1 
            
print('Copied ' + str(train_counter) + ' images to /content/train' )
print('Copied ' + str(validation_counter) + ' images to /content/valid' )

Copied 11954 images to /content/train
Copied 3046 images to /content/valid


In [25]:
!rm export -r
!rm WIDER_test -r
!rm WIDER_train -r
!rm WIDER_val -r
# !rm wider_face_split -r

In [27]:
#Files Count
path, dirs, files = next(os.walk("/content/train/images"))
file_count = len(files)
print(file_count)

40931


# Image testing (Not part of the code)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
image = cv2.imread("/content/1478019952686311006_jpg.rf.a8dc9db36ae4a7e6ecf234f85885a153.jpg")
start_point = (int(0.928- (0.0364/2)), int(0.510 - (0.218/2)))
  
# Ending coordinate, here (220, 220) 
# represents the bottom right corner of rectangle 
end_point = (int(0.928 + (0.0364/2)), int(0.510 + (0.218/2)))
  
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 100
  
# Using cv2.rectangle() method 
# Draw a rectangle with blue line borders of thickness of 2 px 
# image = cv2.rectangle(image, start_point, end_point, color, thickness) 
print(image.shape)
# 0.9286458333333333 * 512
image = cv2.circle(image, (int(0.9286458333333333 * 512),int(0.5108333333333334 * 512)), radius=0, color=(0, 0, 255), thickness=2)
cv2_imshow(image)
344 347 84 138 0 0 0 0 0 0 
539 353 90 117 1 0 0 0 0 0 

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
image = cv2.imread("/content/4_Dancing_Dancing_4_49.jpg")

start_point = (344,347)
end_point = (344 + 84,347 +138)
color = (255, 0, 0) 
thickness = 2
image = cv2.circle(image, (167,301), radius=0, color=(0, 0, 255), thickness=10)

image = cv2.rectangle(image, start_point, end_point, color, thickness) 
cv2_imshow(image)